**BIF7010 | Bioinformatique des structures**

UQAM - Hiver 2019

Amine Remita


**Apprentissage automatique - Partie 3**


* Évaluation de l'apprentissage
* Applications de l'apprentissage automatique en bioinformatique


<a rel="license" href="http://creativecommons.org/licenses/by-sa/4.0/"><img alt="Creative Commons License" style="border-width:0" src="https://i.creativecommons.org/l/by-sa/4.0/88x31.png" /></a>
BIF7101_S3 by Amine Remita is licensed under a Creative Commons Attribution-
ShareAlike 4.0 International License.


Quelques sous-sections et figures sont prises (avec autorisation) à partir des cours de Mohamed Bouguessa Ph.D. (DIC9370) et Ahmed Halioui Ph.D. (BIF7101)

## Rappel 

* Prétraitement de données
* Apprentissage supervisé
* Apprentissage non supervisé

## Évaluation de l’apprentissage

### Utilisation d’un échantillon de test

* La méthode la plus simple pour estimer la qualité d’un algorithme d’apprentissage est de diviser l’ensemble des exemples en deux ensembles indépendants : le premier, noté $A$, est utilisé pour l’apprentissage, le second, noté $T$, sert à mesurer sa qualité.
* $T$ est l’échantillon de test tel que : $S = A ∪ T$ et $A ∩ T = Φ$
* La mesure des erreurs commises par l’algorithme d’apprentissage sur l’ensemble de test T est une estimation de sa qualité

In [ ]:
from sklearn.datasets import load_iris
import numpy as np


iris = load_iris()
X = iris.data
y = np.array(iris.target)


from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.3, random_state=42)

print(X_train.shape)
print(X_test.shape)

### La validation croisée

* Diviser les données d’appentissage $S$ en k sous-échantillons de tailles égales
* Répeter
    * Retenir l’un de ces échantillons ($i$). 
    * Entrainer l’algorithme $C$ sur l’ensemble $S − i$
    * Mesurer le taux d’erreur $R_i(C)$ sur l’ensemble de test $i$
* Recommencer le processus décrit ci-dessus pour chaque échantillon $i$ 
* L’erreur estimée finale est donnée par la moyenne des erreurs mesurées

<img src="figs/cross_validation.png" width="500" />

In [ ]:
from sklearn.model_selection import cross_validate
from sklearn import svm
from pprint import pprint

scoring = ['precision_weighted', 'recall_weighted', 'f1_weighted']

clf = svm.SVC(kernel='linear', C=1, random_state=0)

scores = cross_validate(clf, X, y, scoring=scoring, cv=5, )

pprint(scores)


In [ ]:
from sklearn.model_selection import cross_val_score

clf = svm.SVC(kernel='linear', C=1)

scores = cross_val_score(clf, X, y, cv=10, scoring='f1_weighted')
scores

### Leave-one-out

Lorsque les données disponibles sont très peu nombreuses, il est possible de pousser à l’extrême la méthode de validation croisée en prenant k le nombre total d’exemple diponible (k = n). Dans ce cas, on ne retient à chaque fois qu’un seul exemple pour le test, et on répète l’apprentissage k fois pour tous les autres exemples d’apprentissage.


<img src="figs/leave.png" width="300" />

In [ ]:
from sklearn.model_selection import LeaveOneOut
from sklearn.metrics import accuracy_score

loo = LeaveOneOut()
clf = svm.SVC(kernel='linear', C=1)

scores = list()

for train, test in loo.split(X):
    # apprentissage
    clf.fit(X[train], y[train])
    # prédiction
    y_pred = clf.predict(X[test])
    # calcul score
    scores.append(accuracy_score(y[test], y_pred))

scores = np.array(scores)
print(scores)

### Matrice de confusion

Mesurer la qualité de généralisation du classificateur

![](figs/matrice.png)

In [ ]:
from sklearn.metrics import confusion_matrix

y_true = ["A", "A", "A", "B", "B", "B", "B"]
y_pred = ["A", "B", "A", "A", "B", "B", "B"]

confusion_matrix(y_true, y_pred)



### Calcul des mesures

$ Rappel = \frac{TP}{TP + FN} $
(par rapport aux instances réelles)

$ Précision  = \frac{TP}{TP + FP} $
(par rapport aux instances prédites)

$ F-mesure = \frac{2 (Precision \times Rappel)}{Precision + Rappel)} $

In [ ]:
# https://scikit-learn.org/stable/modules/model_evaluation.html
from sklearn.metrics import recall_score, precision_score, f1_score

y_true = [0, 0, 0, 1, 1, 1, 1]
y_pred = [0, 1, 0, 0, 1, 1, 1]

print("Rappel = {}".format(recall_score(y_true, y_pred)))
print("Précision = {}".format(precision_score(y_true, y_pred)))
print("F-measure = {}".format(f1_score(y_true, y_pred)))

**True Positive Rate**
(ou sensibilité)

$ TPR = \frac{TP}{TP + FN} $
				

**True Negative Rate**
(ou Spécificité)

$ TNR = \frac{TN}{FP + TN} $
				

**False Positive Rate**
(ou 1-Spécificité)

$ FPR = \frac{FP}{FP + TN} $ 
				

**False Negative Rate**

$ FNR = \frac{FN}{TP + FN} $

### Courbe ROC

* ROC : Receiver Operating Characteristic
* Elle met en ralation dans un graphique les taux de faux positifs (en abscisse) et les taux de vraix postifs (en ordonnée)

* Elle est définie pour les problémes de deux classes


<img src="figs/courbe_roc.png" width="400" />

Quelques points importants dans la courbe :

* (FPR, TPR) :
* (0, 0) prédit toujours négatif
* (1, 1) prédit toujours positif
* (0, 1) classification idéale
* Ligne diagonale (ligne de hasard) : classification aléatoire

In [ ]:
import numpy as np
from sklearn.metrics import roc_curve

# exemple de https://scikit-learn.org/stable/modules/generated/sklearn.metrics.roc_curve.html
#y = np.array([1, 1, 2, 2])
#scores = np.array([0.1, 0.4, 0.35, 0.8])
#fpr, tpr, thresholds = roc_curve(y, scores, pos_label=1)
#print(thresholds)


from sklearn.naive_bayes import GaussianNB, MultinomialNB

# Instanciation du classifieur
clf = MultinomialNB()

# Apprentissage/entrainement
clf.fit(X_train, y_train)

# Prédiction 
scores = clf.predict_proba(X_test)

fpr, tpr, thresholds = roc_curve(y_test, scores[:,0], pos_label=0)
print(fpr)
print(tpr)
print(thresholds)



## Applications de l'apprentissage automatique en bioinformatique


<img src="figs/learn_bio2.png" width="800" />



<img src="figs/workflow_data.png" width="800" />

### Utilisation de l’apprentissage supervisé (Jensen et Bateman 2011)

<img src="figs/learn_bio1.png" width="650" />

## Atelier : Prédiction des précurseurs des miARNs 

### Triplet-SVM (Xue et al., 2005)


<img src="figs/triplet_svm_2005.png" width="750" />


### Partie 1 : Prédiction des vrais précurseurs des miARNs  de Homo Sapiens

Fichiers d'entrée :
* **mirna_hsa_fold.fasta** : séquences H. Sapiens de vrais précurseurs de miRNAs avec leurs repliements
* **mirna_hsa_neg_fold.fasta** : séquences H. Sapiens de faux précurseurs de miRNAs avec leurs repliements


1. Calculez le nombres d’occurrences des triplets des séquences avec **calculer_Xu_triplets.pl**

2. Créez des jeux de données compatibles avec scikit-learn (matrice de données ***X*** et vecteur de classes ***y***) en fusionnant les données positives et négatives
    * Deux classes : positive et negative

3. Utilisez Scikit-learn pour l’apprentissage de differents algorithmes en utilisant une validation croisée

### Partie 2 : Prédiction des vrais précurseurs des miARNs de mirRBase

Les séquences négatives des précurseurs peuvent être générées avec plusieurs approches

* Brassage des nucléotides des séquences positives
* Relocalisation de sous-séquences dans les séquences positives
* Utilisation de séquences codantes dans le génome

Refaire le même exercice de la partie 1 avec toutes les données de miRBase et deux jeux de données négatives différents

Fichiers d'entrée :
* **mirbase21_hairpins_fold.fasta** : séquences de vrais précurseurs de tout miRBase
* **mirbase21_hairpins_neg1_fold.fasta** : séquences de faux précurseurs, approche 1
* **mirbase21_hairpins_neg2_fold.fasta** : séquences de faux précurseurs, approche 2

### Partie 3 : Comparaison des précurseurs des animaux et des plantes

À partir du jeux de données construit à partir du fichier **mirbase21_hairpins_fold.fasta**:

1. Générez un jeux de données des précurseurs d'animaux (homme, souris, etc.) et un autre des plantes (Arabidopsis, riz, mais etc.)
    * Deux classes : animal et plante
2. Utilisez Scikit-learn pour l’apprentissage de differents algorithmes en utilisant une validation croisée
3. Est ce que les Triplets peuvent discriminer entre les précurseurs des animaux et des plantes ?
